In [16]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from PIL import Image
import pickle
from sklearn.utils import resample
from matplotlib.pyplot import bar
from skimage.exposure import histogram
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [17]:

# Show the figures / plots inside the notebook
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 


def showHist(img):
    # An "interface" to matplotlib.axes.Axes.hist() method
    plt.figure()
    imgHist = histogram(img, nbins=256)
    
    bar(imgHist[1].astype(np.uint8), imgHist[0], width=0.8, align='center')


In [18]:
def rename_and_convert_images(folder_path):
    # Get all files in the folder
    files = os.listdir(folder_path)
    
    # Filter image files and sort them
    image_files = [f for f in files if f.lower().endswith(('.png', '.bmp', '.jpg', '.jpeg'))]
    image_files.sort()  # Optional: Ensures renaming follows sorted order

    # Process each image
    for idx, file_name in enumerate(image_files, start=1):
        old_path = os.path.join(folder_path, file_name)
        
        # Set new name with index and PNG extension
        new_name = f"{idx}.png"
        new_path = os.path.join(folder_path, new_name)
        
        try:
            # Convert BMP to PNG and resize
            img = cv2.imread(old_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Failed to load image: {file_name}")
                continue
            
            background_pixel = img[0, 0]
            if background_pixel < 128:  # Background is not white (dark)
                # print(f"Inverting background for {file_name}")
                img = cv2.bitwise_not(img)  # Invert the colors

            # Resize to 32x32
            resized_img = cv2.resize(img, (32, 32))
            
            # Save the resized image using PIL for PNG format
            pil_img = Image.fromarray(resized_img)
            pil_img.save(new_path, "PNG")
            
            # Remove the old BMP file
            os.remove(old_path)
        
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
            continue  # Skip to the next file if an error occurs

    print("Renaming, resizing, and conversion completed.")



In [19]:
def extract_hog_features(img):
    # Ensure input is a NumPy array
    if img is None or not isinstance(img, np.ndarray):
        raise ValueError("Invalid input image. Ensure it's loaded correctly.")
    # Resize image to target size
    img = cv2.resize(img, (32, 32))
    # show_images([img])
    
    # Define HOG descriptor parameters
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    block_size = (block_size_in_cells[1] * cell_size[1],
                  block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9
    
    # Initialize HOG descriptor
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    
    # Compute HOG features
    h = hog.compute(img)
    return h.flatten()


In [20]:
def process_images_in_folder(label_folder, label, output_csv="features_with_labels.csv"):
    # Check if the CSV file exists
    file_exists = os.path.isfile(output_csv)
    
    with open(output_csv, "a", newline="") as file:
        writer = csv.writer(file)
        
        # Write header if the file doesn't exist
        if not file_exists:
            num_features = 1764  # Adjust to match your HOG settings
            header = [f"x{i+1}" for i in range(num_features)] + ["y"]
            writer.writerow(header)
            print(f"Created {output_csv} and added header.")
        
        # Iterate through all image files in the label folder
        for file_name in os.listdir(label_folder):
            file_path = os.path.join(label_folder, file_name)
            
            # Skip non-image files
            if not file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                continue
            
            # Load the image
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Use grayscale for HOG
            if img is None:
                print(f"Failed to load image: {file_name}")
                continue
            
            # Extract HOG features
            features = extract_hog_features(img)
            # print(len(features))
            # Append the label and write to the CSV
            writer.writerow(np.append(features, label))
            print(f"Processed: {file_name} with label {label}")

    print(f"Features and labels saved to {output_csv}")


In [ ]:
# # Provide the folder path containing the images
# folder_path = "./dataset/digits_dataset/9"
# rename_and_convert_images(folder_path)


In [ ]:

# label_folder = "./dataset/a_1"
# process_images_in_folder(label_folder,"a_1","a_1_features_with_labels.csv")

# label_folder = "./dataset/a_2"
# process_images_in_folder(label_folder,"a_2","a_2_features_with_labels.csv")

# label_folder = "./dataset/a_4"
# process_images_in_folder(label_folder,"a_4","a_4_features_with_labels.csv")

# label_folder = "./dataset/a_8"
# process_images_in_folder(label_folder,"a_8","a_8_features_with_labels.csv")

# label_folder = "./dataset/a_16"
# process_images_in_folder(label_folder,"a_16","a_16_features_with_labels.csv")

# label_folder = "./dataset/a_32"
# process_images_in_folder(label_folder,"a_32","a_32_features_with_labels.csv")

# label_folder = "./dataset/double_flat"
# process_images_in_folder(label_folder,"double_flat","double_flat_features_with_labels.csv")

# label_folder = "./dataset/double_sharp"
# process_images_in_folder(label_folder,"double_sharp","double_sharp_features_with_labels.csv")

# label_folder = "./dataset/flat"
# process_images_in_folder(label_folder,"flat","flat_features_with_labels.csv")

# label_folder = "./dataset/natural"
# process_images_in_folder(label_folder,"natural","natural_features_with_labels.csv")

# label_folder = "./dataset/sharp"
# process_images_in_folder(label_folder,"sharp","sharp_features_with_labels.csv")

# label_folder = "./dataset/digits_dataset/9"
# process_images_in_folder(label_folder,"9","9_features_with_labels.csv")

In [21]:
# Define a mapping for predictions
label_mapping = {
    1: "digit 1",
    2: "digit 2",
    3: "digit 3",
    4: "digit 4",
    5: "digit 5",
    6: "digit 6",
    7: "digit 7",
    8: "digit 8",
    9: "digit 9",
    11: "a_1",
    12: "a_2",
    14: "a_4",
    18: "a_8",
    16: "a_16",
    32: "a_32",
    33: "double flat",
    34: "double sharp",
    35: "flat",
    36: "natural",
    37: "sharp"
}


In [32]:

# Step 1: Load All CSV Files from a Folder
def load_csvs_from_folder(folder_path):
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    dataframes = [pd.read_csv(csv_file) for csv_file in csv_files]
    return dataframes

# Step 2: Balance Each Dataset to Target Size
def balance_to_avg(dataframes, target_size=800):
    balanced_dataframes = []
    for df in dataframes:
        if len(df) < target_size:
            # Augment smaller datasets by resampling
            augmented_df = resample(
                df, 
                replace=True, 
                n_samples=target_size, 
                random_state=42
            )
            balanced_dataframes.append(augmented_df)
        else:
            # Downsample larger datasets
            downsampled_df = df.sample(n=target_size, random_state=42)
            balanced_dataframes.append(downsampled_df)
    return pd.concat(balanced_dataframes)

# Step 3: Train and Evaluate Classifiers
def train_classifiers(data):
    # Separate features and labels
    X = data.iloc[:, :-1].values  # All columns except the last one
    y = data.iloc[:, -1].values   # Last column as target
    
    # Convert labels to numeric if necessary
    if y.dtype == 'object':
        y = y.astype(int)
    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("X_train")
    print(X_train)
    # Define classifiers
    # svc = SVC(kernel='linear', probability=True, random_state=42)
    svc = SVC(kernel='poly',degree=2,C=0.1,gamma=0.1,probability=True,random_state=42) 
    rf = RandomForestClassifier(random_state=42)
    log_reg = LogisticRegression(random_state=42, max_iter=1000)  # Added Logistic Regression

    # Voting classifier
    voting_clf = VotingClassifier(
        estimators=[('SVM', svc), ('RF', rf)],
        voting='soft'
    )

    # Train classifiers
    voting_clf.fit(X_train, y_train)
    svc.fit(X_train, y_train)
    rf.fit(X_train, y_train)
    log_reg.fit(X_train, y_train)
    
    # Evaluate classifiers
    voting_preds = voting_clf.predict(X_test)
    svc_preds = svc.predict(X_test)
    rf_preds = rf.predict(X_test)
    log_reg_preds = log_reg.predict(X_test)

    # Print accuracies
    print("Voting Classifier Accuracy:", accuracy_score(y_test, voting_preds))
    print("SVM Accuracy:", accuracy_score(y_test, svc_preds))
    print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
    print("Logistic Regression Accuracy:", accuracy_score(y_test, log_reg_preds))

    return voting_clf,svc,rf,log_reg,scaler

# Specify the folder containing your CSV files
folder_path = "./csvs"  # Replace with your folder path if different

# Load, balance, and combine the datasets
dataframes = load_csvs_from_folder(folder_path)
balanced_data = balance_to_avg(dataframes, target_size=500)

# Train and evaluate classifiers
voting_clf,svc,rf,log_reg,scaler=train_classifiers(balanced_data)


X_train
[[-0.3286457  -0.29321716 -0.25324788 ... -0.25281309 -0.28537911
  -0.52145781]
 [-0.39235534 -0.29321716 -0.27485905 ... -0.25281309 -0.25386744
   1.92391937]
 [-0.39235534 -0.29321716 -0.27485905 ... -0.25281309 -0.28537911
  -0.11374549]
 ...
 [-0.39235534 -0.29321716 -0.27485905 ... -0.25281309 -0.28537911
  -0.52145781]
 [-0.39235534 -0.29321716 -0.27485905 ... -0.25281309 -0.28537911
   0.27384111]
 [-0.39235534 -0.29321716 -0.27485905 ... -0.25281309 -0.28537911
  -0.52145781]]
Voting Classifier Accuracy: 0.9835
SVM Accuracy: 0.984
Random Forest Accuracy: 0.962
Logistic Regression Accuracy: 0.9675


In [33]:
# 2. Define classifiers
clf1 = LinearDiscriminantAnalysis()
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
clf4= SVC(kernel='poly',degree=2,C=0.1,gamma=0.1) 

# Specify the folder containing your CSV files
folder_path = "./csvs"  # Replace with your folder path if different

# Load, balance, and combine the datasets
dataframes = load_csvs_from_folder(folder_path)
balanced_data = balance_to_avg(dataframes, target_size=500)
classifiers = [('LDA', clf1), ('RF', clf2), ('GNB', clf3),('SVC', clf4)]
X = balanced_data.iloc[:, :-1].values  # All columns except the last one
y = balanced_data.iloc[:, -1].values   # Last column as target

# Convert labels to numeric if necessary
if y.dtype == 'object':
    y = y.astype(int)
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 3. Train and evaluate each classifier individually using cross-validation
for name, clf in classifiers:
    scores = cross_val_score(clf, X, y, cv=5)  
    print(f"{name} Accuracy: {np.mean(scores):.4f}")

# 4. Hard voting classifier
eclf1 = VotingClassifier(estimators=classifiers, voting='hard')

# 5. Evaluate the hard voting classifier using cross-validation
scores = cross_val_score(eclf1, X, y, cv=5)  
print(f"Hard Voting Classifier Accuracy: {np.mean(scores):.4f}")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [23]:

#Split data into test and validation sets
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# Step 2: Define the parameter grid for GridSearchCV
param_grid = {
    'degree': [1,2],  # Test different polynomial degrees
    'C': [0.01,0.1, 1],  # Test different values for regularization parameter C
    'gamma': [0.01,0.1, 1]  # Test different gamma values
}

# Step 3: Create the SVM model
svm_model = SVC(kernel='poly')

# Step 4: Use GridSearchCV for parameter tuning
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_val, y_val)

# Step 5: Get the best parameters and retrain the model
best_params = grid_search.best_params_
best_svm_model = SVC(kernel='poly', **best_params)
best_svm_model.fit(X_train, y_train)  # Retrain on the entire X_train

# Step 6: Evaluate the final model on the test set
test_accuracy = best_svm_model.score(X_test, y_test)

print("Best Parameters:", best_params)
print("Test Accuracy:",test_accuracy)

Best Parameters: {'C': 0.01, 'degree': 2, 'gamma': 0.1}
Test Accuracy: 0.984


In [34]:

# Save models and scaler to .pkl files
with open('./models/voting_clf.pkl', 'wb') as f:
    pickle.dump(voting_clf, f)

with open('./models/svc.pkl', 'wb') as f:
    pickle.dump(svc, f)

with open('./models/rf.pkl', 'wb') as f:
    pickle.dump(rf, f)

with open('./models/log_reg.pkl', 'wb') as f:
    pickle.dump(log_reg, f)

with open('./models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Models and scaler saved successfully using pickle!")


Models and scaler saved successfully using pickle!


In [35]:
# Load models and scaler from .pkl files
with open('./models/voting_clf.pkl', 'rb') as f:
    voting_clf = pickle.load(f)

with open('./models/svc.pkl', 'rb') as f:
    svc = pickle.load(f)

with open('./models/rf.pkl', 'rb') as f:
    rf = pickle.load(f)

with open('./models/log_reg.pkl', 'rb') as f:
    log_reg = pickle.load(f)

with open('./models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

print("Models and scaler loaded successfully using pickle!")


Models and scaler loaded successfully using pickle!


In [40]:
# Test on a new input image
def predict_image(image_path, voting_clf, svc, rf, log_reg, scaler):
     # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Use grayscale for HOG
    if img is None:
        print(f"Failed to load image: {image_path}")
        
    # Preprocess the image and extract features
    features = extract_hog_features(img)

    # Standardize the features using the same scaler as during training
    features = scaler.transform([features])

    # Predict using each classifier
    voting_prediction = voting_clf.predict(features)[0]
    svc_prediction = svc.predict(features)[0]
    rf_prediction = rf.predict(features)[0]
    log_reg_prediction = log_reg.predict(features)[0]

    return {
        "Voting Classifier": voting_prediction,
        "SVM": svc_prediction,
        "Random Forest": rf_prediction,
        "Logisitc Regression": log_reg_prediction,
    }

image_path = "test_image.png"  # Replace with your test image path
predictions = predict_image(image_path, voting_clf, svc, rf,log_reg, scaler)

print("\nPredictions for the input image:")
for classifier, prediction in predictions.items():
    print(f"{classifier}: {prediction}")



Predictions for the input image:
Voting Classifier: 32
SVM: 32
Random Forest: 16
Logisitc Regression: 16
